# K-Nearest-Neighbors (KNN)


## Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

#Importing the data
## About the data

<img src='https://images.squarespace-cdn.com/content/v1/5c36736d9d5abb8b44c6b7f7/1586468701075-G14EXPZE2VNMJFKDCQ0T/rook+and+king+logo.jpg?format=500w' align="center"><br>
  Chess End-Game -- King+Rook versus King+Pawn on a7
  (usually abbreviated KRKPA7).  The pawn on a7 means it is one square
  away from queening.  It is the King+Rook's side (white) to move.

  

In [2]:
king_data=pd.read_excel('/content/King-RookvsKing-Pawn.xlsx')

In [3]:
king_data.head(10)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Column26,Column27,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35,Column36,Column37
0,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
1,f,f,f,f,t,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
2,f,f,f,f,t,f,t,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
3,f,f,f,f,f,f,f,f,t,f,f,f,l,f,n,f,f,t,f,f,f,t,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
4,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,t,f,t,f,f,f,f,f,f,f,t,t,n,won
5,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,t,f,t,f,f,f,t,f,f,f,t,t,n,won
6,f,f,f,f,f,f,f,f,t,f,f,f,l,f,n,f,f,t,f,f,f,t,f,t,f,t,f,f,f,f,f,f,f,t,t,n,won
7,f,f,f,f,t,f,f,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
8,f,f,f,f,f,f,f,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,t,f,f,t,t,n,won
9,f,f,f,f,f,f,t,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won


In [4]:
king_data.columns

Index(['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6',
       'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12',
       'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18',
       'Column19', 'Column20', 'Column21', 'Column22', 'Column23', 'Column24',
       'Column25', 'Column26', 'Column27', 'Column28', 'Column29', 'Column30',
       'Column31', 'Column32', 'Column33', 'Column34', 'Column35', 'Column36',
       'Column37'],
      dtype='object')

In [5]:
object_col=['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6',
       'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12',
       'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18',
       'Column19', 'Column20', 'Column21', 'Column22', 'Column23', 'Column24',
       'Column25', 'Column26', 'Column27', 'Column28', 'Column29', 'Column30',
       'Column31', 'Column32', 'Column33', 'Column34', 'Column35', 'Column36'
       ]

Droping NaN values

In [6]:
king_data.dropna(axis=0)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Column26,Column27,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35,Column36,Column37
0,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
1,f,f,f,f,t,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
2,f,f,f,f,t,f,t,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
3,f,f,f,f,f,f,f,f,t,f,f,f,l,f,n,f,f,t,f,f,f,t,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won
4,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,t,f,t,f,f,f,f,f,f,f,t,t,n,won
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3191,t,f,f,f,f,f,t,f,f,f,f,t,l,f,b,f,f,t,f,f,f,f,t,t,f,t,f,f,t,f,f,t,f,t,f,n,nowin
3192,t,f,f,f,f,f,t,f,f,f,f,t,l,f,b,f,f,t,f,f,f,f,t,f,f,t,f,f,t,f,f,t,f,t,f,n,nowin
3193,t,f,f,f,f,f,t,f,f,f,f,t,l,f,n,f,f,t,f,f,f,f,t,f,f,t,f,f,t,f,f,t,f,t,f,n,nowin
3194,t,f,t,f,f,f,t,f,f,f,f,t,l,t,b,t,f,t,f,f,f,f,f,t,f,t,f,f,t,f,f,t,f,f,f,n,nowin


In [7]:
print(f'{king_data.Column37.unique()}')

['won' 'nowin']


In [8]:
king_data['Column37']=king_data['Column37'].apply({'won':2,'nowin':1}.get)

In [9]:
king_data.head(10)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Column26,Column27,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35,Column36,Column37
0,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2
1,f,f,f,f,t,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2
2,f,f,f,f,t,f,t,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2
3,f,f,f,f,f,f,f,f,t,f,f,f,l,f,n,f,f,t,f,f,f,t,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2
4,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,t,f,t,f,f,f,f,f,f,f,t,t,n,2
5,f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,t,f,t,f,f,f,t,f,f,f,t,t,n,2
6,f,f,f,f,f,f,f,f,t,f,f,f,l,f,n,f,f,t,f,f,f,t,f,t,f,t,f,f,f,f,f,f,f,t,t,n,2
7,f,f,f,f,t,f,f,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2
8,f,f,f,f,f,f,f,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,t,f,f,t,t,n,2
9,f,f,f,f,f,f,t,f,f,f,t,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,2


## Spliting the data.

In [10]:
X=king_data.copy()
y=X.pop('Column37')

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.8,train_size=0.2)


In [11]:
y_test.head(10)

215     2
258     2
1228    1
2616    1
1859    2
758     2
2690    1
1404    1
420     2
2966    1
Name: Column37, dtype: int64

## Training and predicting the model using KNN algo.



In [12]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

one_hot_train=pd.DataFrame(OH_encoder.fit_transform(X_train[object_col]))
one_hot_test=pd.DataFrame(OH_encoder.transform(X_test[object_col]))
one_hot_train.index = X_train.index
one_hot_test.index = X_test.index

In [13]:
num_X_train = X_train.drop(object_col, axis=1)
num_X_valid = X_test.drop(object_col, axis=1)

In [14]:
OH_X_train = pd.concat([num_X_train, one_hot_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, one_hot_test], axis=1)

In [15]:
y_train.unique()

array([2, 1])

In [16]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import GridSearchCV
"""#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
#Fit the model
best_model = clf.fit(OH_X_train,y_train)
#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])"""

"#List Hyperparameters that we want to tune.\nleaf_size = list(range(1,50))\nn_neighbors = list(range(1,30))\np=[1,2]\n#Convert to dictionary\nhyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)\n#Create new KNN object\nknn_2 = KNeighborsClassifier()\n#Use GridSearch\nclf = GridSearchCV(knn_2, hyperparameters, cv=10)\n#Fit the model\nbest_model = clf.fit(OH_X_train,y_train)\n#Print The value of best Hyperparameters\nprint('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])\nprint('Best p:', best_model.best_estimator_.get_params()['p'])\nprint('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])"

In [17]:

"""
model=KNeighborsClassifier(n_neighbors=14,
          leaf_size=27,
          algorithm='brute',
          weights='distance',
          p=1
          )

model.fit(OH_X_train,y_train)"""

"\nmodel=KNeighborsClassifier(n_neighbors=14,\n          leaf_size=27,\n          algorithm='brute',\n          weights='distance',\n          p=1\n          )\n\nmodel.fit(OH_X_train,y_train)"

In [18]:
"""pred=model.predict(OH_X_valid)"""

'pred=model.predict(OH_X_valid)'

## Checking model accuracy.


In [19]:
"""from sklearn.metrics import accuracy_score

x=accuracy_score(y_test,pred)
print(f'the accuracy score is {x}')"""

"from sklearn.metrics import accuracy_score\n\nx=accuracy_score(y_test,pred)\nprint(f'the accuracy score is {x}')"

## Using confusion matric

In [20]:
"""from sklearn.metrics import confusion_matrix

cm= confusion_matrix(y_test,pred)"""

'from sklearn.metrics import confusion_matrix\n\ncm= confusion_matrix(y_test,pred)'

In [21]:
"""print(cm)"""

'print(cm)'

# <B> Decision Tree Classifier

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


In [23]:
#List Hyperparameters that we want to tune.
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)]
min_samples_split = [2, 6, 10]
min_samples_leaf= list(range(1, 10))
criterion= ["gini", "entropy"]
#Convert to dictionary
hyperparameters = dict( max_features=max_features, max_depth=max_depth
                       , min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                       criterion=criterion)
#Create new DTC object
dtc_2 = DecisionTreeClassifier()
#Use GridSearch
clf = GridSearchCV(dtc_2, hyperparameters, cv=10, verbose =1)
#Fit the model
best_model = clf.fit(OH_X_train,y_train)
#Print The value of best Hyperparameters
print(best_model.best_estimator_.get_params())


Fitting 10 folds for each of 1296 candidates, totalling 12960 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 70, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': 'deprecated', 'random_state': None, 'splitter': 'best'}


[Parallel(n_jobs=1)]: Done 12960 out of 12960 | elapsed:   42.1s finished


In [162]:
model=DecisionTreeClassifier(ccp_alpha= 0.0, class_weight= None, criterion= 'entropy',
                             max_depth= 70, max_features= 'auto', max_leaf_nodes= None,
                             min_impurity_decrease= 0.0, min_impurity_split= None, 
                             min_samples_leaf= 1, min_samples_split= 2, min_weight_fraction_leaf= 0.0,
                             presort= 'deprecated', random_state= 6, splitter= 'best'
          )

In [163]:
model.fit(OH_X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=70, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=6, splitter='best')

In [165]:
pred=model.predict(OH_X_valid)
x=accuracy_score(y_test,pred)
print(f'the accuracy score is {x*100}')

the accuracy score is 96.40203363316387
